In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [13]:
######### NEURAL NETWORK MODEL ###########

def nn_model(t, time_series, hidden_width, epochs, title, save_fig=False):
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.cuda.current_device())
    print(torch.cuda.device(0))
    print(torch.cuda.get_device_name(0))
    
    #Our input dimension is 1, and each entry in the tensor above is a datapoint for training. So, the length of 
    #the tensor/numpy array should be the batch size:
    n_input, n_hidden, n_output = 1, hidden_width, 1
    batch_size = 10
    
    #Turn into tensor
    t_tensor = torch.from_numpy(np.reshape(t, (-1, n_input))).to(torch.float32)
    y_tensor = torch.from_numpy(np.reshape(time_series, (-1, n_output))).to(torch.float32)

    dataset = TensorDataset(t_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    dataset_size = len(dataloader.dataset)
    
    learning_rate = 0.001
    
    plt.ylabel('y')
    plt.xlabel('t')
    plt.title("Given Time Series")
    plt.plot(t_tensor, y_tensor, marker='.', linestyle='none')
    plt.show()

    model = nn.Sequential(nn.Linear(n_input, n_hidden), nn.ReLU(), nn.Linear(n_hidden, n_output))
    print(model)

    loss_function = nn.MSELoss(reduction="sum")
    print(loss_function)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    losses = [] # For plotting purposes
    print("---------------------------------------------------------")
    print("START TRAINING")
    print(f"HIDDEN LAYER WIDTH: {hidden_width}")
    for epoch in range(epochs):
        #print(f"Epoch {epoch}\n------------------------")
        
        losses_epoch = []
        for id_batch, (t_batch, y_batch) in enumerate(dataloader):
            # Forward pass
            y_pred = model(t_batch)
            
            # Calculate the loss
            loss = loss_function(y_pred, y_batch)
            losses_epoch.append(loss.item())
            
            # We set the gradients to zero, as PyTorch accumulates gradients on subsequent backward passes.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        losses.append(np.mean(losses_epoch))
    print("END TRAINING")
    
    y_pred_final = model(t_tensor)
    rel_error_final = relative_error(y_pred_final.detach().numpy(), y_tensor.detach().numpy())
    
    with torch.no_grad():
        plt.title(f"{title}")
            
        plt.plot(t_tensor, y_tensor, label="True") #, marker='.', linestyle='none')
        plt.plot(t_tensor, y_pred_final, label=f"NN Pred. Width={hidden_width}") #, marker='.', linestyle='none', color='orange')
        plt.legend()
        if save_fig:
            plt.savefig(savefigs_loc_nn+"_TRAINING_"+title+f"_hw_{hidden_width}__epochs_{epochs}__rel_error_{rel_error_final:.4}_.png")
        plt.show()
    
    plt.plot(losses)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title(title+f"_width={hidden_width}_"+"lr = %f"%(learning_rate))
    if save_fig:
        plt.savefig(savefigs_loc_nn+"_LOSS_"+title+f"_hw_{hidden_width}__epochs_{epochs}__rel_error_{rel_error_final:.4}_.png")
    plt.show()
    
    print(f"Relative error = {rel_error_final:.4}")
    print("--------------------------------------")
    return model, rel_error_final, y_pred_final, y_tensor

In [15]:
#### CONSTRUCT MULTUPLE NEURAL NETWORKS WITH DIFFERENT WIDTHS ####

def nn_models(hidden_widths, t, time_series, epochs, title, save_fig=False):
    models_dict = {}
    rel_err_dict = {}
    final_preds_dict = {}
    ts_tensor_dict={}
    
    for hw in hidden_widths:
        mod, rel_err, y_pred_final, ts_tensor = nn_model(t, time_series, hw, epochs, title, save_fig)
        models_dict[hw]=mod
        rel_err_dict[hw]=rel_err
        
        final_preds_dict[hw] = y_pred_final
        ts_tensor_dict[hw] = ts_tensor
    
    plt.plot(t, ts_tensor_dict[hidden_widths[0]].detach().numpy(), label="True")
    for hw in hidden_widths:
        plt.plot(t, final_preds_dict[hw].detach().numpy(), label=f"NN pred. Width={hw}")
    plt.legend()
    plt.title(title)
    if save_fig:
        plt.savefig(savefigs_loc_nn+"_TRAINING_ALL_HWs__"+title+f"__epochs_{epochs}_.png")
    plt.show()
    
    
    x = y = list(range(hidden_widths[-1]))
    idx = rel_err_dict.keys()
    new_y = [rel_err_dict[i] for i in idx]
    plt.plot(range(len(idx)), new_y, 'o-')
    plt.xticks(range(len(idx)),idx)
    plt.xlabel("Hidden layer width")
    plt.ylabel("Training, Relative error")
    plt.legend()
    if save_fig:
        plt.savefig(savefigs_loc_nn+"_REL_ERRORS_vs_HWs__"+title+f"__epochs_{epochs}_.png")
    plt.show()
    
    best_width = min(rel_err_dict, key=rel_err_dict.get)
    print("Best width: ", best_width)
    return models_dict, best_width

In [16]:
### TEST NEURAL NETWORK ON TEST DATA, ON TRAINING INTERVAL OR FORWARD IN TIME ###

def nn_test(model,hidden_width, test_time, test_time_series, title, forward_time=False, save_fig=False):
    n_input, n_output = 1, 1
    test_time_tensor = torch.from_numpy(np.reshape(test_time, (-1, n_input))).to(torch.float32)
    test_time_series_tensor = torch.from_numpy(np.reshape(test_time_series, (-1, n_output))).to(torch.float32)
    
    pred = model(test_time_tensor)
    
    plt.title(title)
    
    if forward_time:
        middle_index = test_time.shape[0]//2
        forward_rel_err = relative_error(pred.detach().numpy()[middle_index:], test_time_series_tensor.detach().numpy()[middle_index:])
          
        plt.plot(test_time_tensor.detach().numpy(), test_time_series_tensor.detach().numpy(), label="True")
        plt.plot(test_time_tensor.detach().numpy(), pred.detach().numpy(), label=f"Pred. Width={hidden_width}")
        plt.legend()
        if save_fig:
            plt.savefig(savefigs_loc_nn+"_FORWARD_"+title+f"_hw_{hidden_width}__forward_rel_error_{forward_rel_err:.4}_.png")
        
        print("Forward, relative error (on future time points): ", forward_rel_err)
    else:
        test_rel_err = relative_error(pred.detach().numpy(), test_time_series_tensor.detach().numpy())
        
        plt.plot(test_time_tensor.detach().numpy(), tesla_2021_data.to_numpy(), color="gray",alpha=0.5, label="Original")
        
        plt.plot(test_time_tensor.detach().numpy(), test_time_series_tensor.detach().numpy(), label="Smoothed")
        plt.plot(test_time_tensor.detach().numpy(), pred.detach().numpy(), label=f"Pred. Width={hidden_width}")
        plt.legend()
        if save_fig:
            plt.savefig(savefigs_loc_nn+"_TESTING_"+title+f"_hw_{hidden_width}__forward_rel_error_{test_rel_err:.4}_.png")

        
        print("Testing (same interval), relative error: ", test_rel_err)
    
    plt.show()
    return

In [1]:
def relative_error(prediction, true):
    mse = np.mean(np.power(prediction-true,2))
    print("mse: ", mse)
    true_size = np.mean(np.power(true,2))
    print("true size: ", true_size)
    return mse/true_size